In [ ]:
import numpy as np

In [ ]:
def FDependiente(k,F):
    F=k*F
    return F

def Paralelo(R1,R2):
    Req=((1/R1)+(1/R2))**(-1)
    return Req

def Serie(R1,R2):
    Req=R1+R2
    return Req

def Ohm(R=False,V=False,I=False):
    if R==False:
        R=V/I
        return R
    
    if V==False:
        V=I*R
        return V
    
    if I==False:
        I=V/R
        return I
    
def Potencia(R=False,V=False,I=False):
    if R==False:
        P=I*V
        return P
    
    if V==False:
        P=(I**2)*R
        return P
    
    if I==False:
        P=(V**2)/R
        return P
    
def DivisionCorriente(Rp,Rc,I):
    Ip=(I*Rc)/(Rc+Rp)
    return Ip

def DivisionVoltaje(Rp,Rc,V):
    Vp=(V*Rp)/(Rp+Rc)
    return Vp

In [ ]:
def Mallas(Imalla,Rdict,Mdict,SM=False,FD=False):
    """Input: 
    -Imalla: Corrientes de Malla del circuito en una lista, ejemplo: ['i1','i2','i3']
    
    -Rdict: Diccionario de las Resistencias del circuito donde se coloca su tag y su valor, ejemplo: {'R1':3,'R2':2}
    
    -Mdict: Diccionario con la Descripción del circuito por malla donde se coloca el numero de la malla y dentro de este otro diccionario con la fuente de voltaje (si no hay colocar 0) y las Resistencias de malla, ejemplo:
    {1:{'V':5,'R':['R1','R5']}}
    
    -SM: Diccionario con descripción de la supermalla (False por defecto), en caso de tener se escribe True y se coloca la corriente de la fuente, las mallas con las que se comparten y la dirección de la corriente (1:Con Corriente,-1: ContraCorriente), Ejemplo: {True:{'I':4,'M':{3:1,4:1}}}
    
    Output: 
    -Diccionario con los tags de las corrientes de malla y sus valores, Ejemplo {'i1':5.783}
    -Matrix de Resistencias A
    -Matrix de Entradas b"""
    
    Rpropia=ResitenciaPropia(Mdict,Rdict,FD)
    Rmutua=ResistenciaMutua(Mdict,Rdict,FD)
    b=Input(Mdict,SM)
    A=Resistencias(Mdict,Rpropia,Rmutua,SM)
    Output=np.dot(np.linalg.inv(A),b.T)
    Output_Dict={}
    for i in range(len(Output)):
        Output_Dict[Imalla[i]]=round(float(Output[i]),3)
    return Output_Dict,A,b

In [ ]:
def Resistencias(Mdict,Rpropia,Rmutua,SM=False):
    if SM==False:  
        R=[]
        for i in range(len(Mdict)):
            r=[]
            for j in range(len(Mdict)):
                if i==j:
                    r.append(Rpropia[i+1])
                else:
                    r.append(Rmutua[i+1][j+1])
            R.append(r)
        A=np.array(R)
        
    else:
        A=0
        position=1e100
        for i in SM[True]['M']:
            if i<position:
                position=i
            if i>position:
                shared=i
        R=[]
        for i in range(len(Mdict)):
            r=[]
            for j in range(len(Mdict)):
                if i+1==position:
                    if i==j:
                        r.append(Rpropia[i+1])
                    elif j+1==shared:
                        r.append(Rpropia[shared])
                    else:
                        if Rmutua[i+1][j+1]==0:
                            r.append(Rmutua[shared][j+1])
                        else:
                            r.append(Rmutua[i+1][j+1])
                elif i+1==shared:
                    if j+1==position:
                        r.append(SM[True]['M'][position])
                    elif j+1==shared:
                        r.append(SM[True]['M'][shared])
                    else:
                        r.append(0)
                else:
                    if i==j:
                        r.append(Rpropia[i+1])
                    else:
                        r.append(Rmutua[i+1][j+1])
            R.append(r)
        A=np.array(R)
    return A

In [ ]:
def Input(Mdict,SM=False):
    if SM==False:
        V=[]
        for i in range(len(Mdict)):
            v=Mdict[i+1]['V']
            V.append(v)
        b=np.array([V])
        return b
    else:
        V=[]
        p=0
        for i in SM[True]['M'].keys():
            if p<i:
                p=i
        for i in range(len(Mdict)):
            if i+1!=p:
                v=Mdict[i+1]['V']
                V.append(v)
            else:
                V.append(SM[True]['I'])
        b=np.array([V])
    return b

In [ ]:
def ResitenciaPropia(Mdict,Rdict,FD):
    Rpropia={}
    for i in range(len(Mdict)):
        t=0
        R=Mdict[i+1]['R']
        for r in R:
            t=t+Rdict[r]
            Rpropia[i+1]=t
            
    if FD==True:
        f={}
        for i in range(len(Mdict)):
            if 'D' in Mdict[i+1].keys():
                for F in Mdict[i+1]['D']:
                    if Rdict[F]['D']!=i+1:
                        continue
                    f[i+1]=Mdict[i+1]['D']

        for j in f.keys():
            for F in f[j]:
                Rpropia[j]+=Rdict[F]['k']
        
    
    return Rpropia

In [ ]:
def ResistenciaMutua(Mdict,Rdict, FD):
    Rmutua={}
    for i in range(len(Mdict)):
        Rmutua[i+1]={}
        Ri=Mdict[i+1]['R']
        for j in range(len(Mdict)):
            if i==j:
                Rmutua[i+1][j+1]=0
            t=0
            Rmutua[i+1][j+1]=0
            Rj=Mdict[j+1]['R']
            for r in Rj:
                if r in Ri:
                    t=t+Rdict[r]
                    Rmutua[i+1][j+1]=-t
    if FD==True:
        f={}
        for i in range(len(Mdict)):
            if 'D' in Mdict[i+1].keys():
                for F in Mdict[i+1]['D']:
                    if Rdict[F]['D']==i+1:
                        continue
                    f[i+1]=Mdict[i+1]['D']
        
        for j in f.keys():
            for F in f[j]:
                Rmutua[j][Rdict[F]['D']]+=Rdict[F]['k']
        
    return Rmutua